In [6]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [7]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"

In [9]:
df=pd.read_csv('/content/Dataset/train.dat' ,delimiter='\t',header=None,names=["lables","text"])

In [10]:
df.head()

,lables,text
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14438 entries, 0 to 14437
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lables  14438 non-null  int64 
 1   text    14438 non-null  object
dtypes: int64(1), object(1)
memory usage: 225.7+ KB


In [12]:
df["lables"]=df["lables"]-1

In [13]:
df["text"][1]

'Renal abscess in children. Three cases of renal abscesses in children are described to illustrate the variable presenting features. An additional 23 pediatric cases, reported over the past ten years, were reviewed for clinical features and therapy. Fever, loin pain, and leukocytosis were common presenting features, but less than half of all abscesses were associated with either an abnormal urinalysis or a positive urine culture. The presenting features were sometimes confused with appendicitis, peritonitis, or a Wilms tumor. An organism was identified in 17 cases--Escherichia coli in 9 children and Staphylococcus aureus in 8 children. The majority of E. coli infections occurred in girls and the majority of S. aureus infections occurred in boys. Reflux was documented in 5 patients, and 2 children had a possible extrarenal source of infection. Antibiotics alone produced a cure in 10 children (38%), but 16 children (62%) required a surgical procedure. '

In [14]:
df["lables"].unique()

array([3, 4, 1, 0, 2])

In [15]:
text = df.text.values
labels = df.lables.values

In [16]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True,truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
print(' Original: ', text[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(text[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

 Original:  Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left ante

In [18]:
max_len = 0

# For every sentence...
for sent in text:
    if len(sent) > 512:
        sent = sent[:512]

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  188


In [19]:
input_ids = []
attention_masks = []
# For every tweet...
for tweet in text:

    # `encode_plus` will:
    #   (1) Tokenize the sentence.

    encoded_dict = tokenizer.encode_plus(
                        tweet,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',
                        truncation=True   # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', text[0])
print('Token IDs:', len(input_ids[0]))


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). Cardiogenic shock was present in eight patients with infarction of the left anter

In [20]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
#val_size = int(0.2 * len(dataset))
val_size = len(dataset)  - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

11,550 training samples
2,888 validation samples


In [21]:
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [22]:
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 5, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [25]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the device using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        print(b_labels)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
tensor([4, 4, 4, 3, 2, 1, 3, 3, 2, 0, 1, 0, 4, 3, 3, 3, 0, 0, 3, 2, 1, 0, 3, 3,
        4, 4, 4, 3, 3, 3, 3, 3])
tensor([0, 2, 3, 0, 0, 0, 4, 1, 4, 1, 3, 4, 1, 4, 2, 0, 4, 4, 1, 1, 3, 0, 0, 3,
        0, 2, 2, 3, 1, 4, 2, 0])
tensor([1, 0, 4, 4, 3, 4, 4, 1, 4, 3, 3, 4, 0, 1, 1, 4, 0, 0, 0, 4, 2, 0, 3, 3,
        4, 3, 1, 4, 2, 4, 3, 3])
tensor([3, 1, 2, 4, 0, 2, 4, 4, 2, 3, 1, 0, 2, 2, 0, 0, 4, 3, 2, 4, 0, 4, 1, 4,
        3, 3, 2, 3, 4, 4, 4, 0])
tensor([3, 0, 3, 4, 4, 4, 1, 0, 1, 2, 3, 0, 0, 4, 4, 4, 3, 3, 4, 4, 0, 4, 1, 3,
        4, 3, 4, 4, 1, 0, 3, 4])
tensor([0, 4, 1, 0, 4, 0, 4, 4, 0, 3, 4, 0, 0, 3, 3, 0, 3, 4, 0, 3, 0, 2, 2, 3,
        4, 3, 3, 0, 0, 0, 2, 4])
tensor([4, 4, 4, 3, 2, 0, 1, 1, 1, 4, 4, 4, 4, 1, 0, 2, 0, 4, 3, 4, 3, 2, 3, 0,
        2, 0, 0, 3, 2, 3, 3, 0])
tensor([0, 1, 2, 1, 4, 4, 4, 0, 0, 0, 3, 4, 0, 0, 2, 3, 4, 4, 2, 1, 3, 1, 3, 4,
        2, 4, 0, 4, 2, 2, 0, 2])
tensor([4, 4, 1, 3, 1, 4, 3, 0, 4, 3, 2, 0, 2, 3, 3, 

In [23]:
#print("yes")

In [29]:
model.save_pretrained('results/model/')

In [67]:
# df_test = pd.read_csv('/content/Dataset/test.dat',delimiter='\t',header=None,names=["text"])
# test_data = df_test['text'].values

In [66]:
# test_input_ids = []
# test_attention_masks = []
# for tweet in test_data:
#     encoded_dict = tokenizer.encode_plus(
#                         tweet,
#                         add_special_tokens = True,
#                         max_length = max_len,
#                         pad_to_max_length = True,
#                         return_attention_mask = True,
#                         return_tensors = 'pt',
#                         truncation=True
#                    )
#     test_input_ids.append(encoded_dict['input_ids'])
#     test_attention_masks.append(encoded_dict['attention_mask'])
# test_input_ids = torch.cat(test_input_ids, dim=0)
# test_attention_masks = torch.cat(test_attention_masks, dim=0)

In [65]:
# test_dataset = TensorDataset(test_input_ids, test_attention_masks)
# test_dataloader = DataLoader(
#             test_dataset, # The validation samples.
#             sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )

In [64]:
# predictions = []
# for batch in test_dataloader:
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         with torch.no_grad():
#             output= model(b_input_ids,
#                                    token_type_ids=None,
#                                    attention_mask=b_input_mask)
#             logits = output.logits
#             logits = logits.detach().cpu().numpy()
#             pred_flat = np.argmax(logits, axis=1).flatten()

#             predictions.extend(list(pred_flat))

In [63]:
# tokenizer.save_pretrained('results/tokenizer/')

In [62]:
# tokenizer1 = BertTokenizer.from_pretrained("/content/results/tokenizer", local_files_only=True)

In [61]:
#s="""Renal abscess in children. Three cases of renal abscesses in children are described to illustrate the variable presenting features. An additional 23 pediatric cases, reported over the past ten years, were reviewed for clinical features and therapy. Fever, loin pain, and leukocytosis were common presenting features, but less than half of all abscesses were associated with either an abnormal urinalysis or a positive urine culture. The presenting features were sometimes confused with appendicitis, peritonitis, or a Wilms tumor. An organism was identified in 17 cases--Escherichia coli in 9 children and Staphylococcus aureus in 8 children. The majority of E. coli infections occurred in girls and the majority of S. aureus infections occurred in boys. Reflux was documented in 5 patients, and 2 children had a possible extrarenal source of infection. Antibiotics alone produced a cure in 10 children (38%), but 16 children (62%) required a surgical procedure."""

In [60]:

# encoded_dict = tokenizer1.encode_plus(
#                         s,
#                         add_special_tokens = True,
#                         max_length = max_len,
#                         pad_to_max_length = True,
#                         return_attention_mask = True,
#                         return_tensors = 'pt',
#                         truncation=True
#                    )

In [59]:
# encoded_dict["input_ids"]

In [58]:
# from transformers import TextClassificationPipeline


# pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer1, return_all_scores=True)
# # outputs a list of dicts like [[{'label': 'NEGATIVE', 'score': 0.0001223755971295759},  {'label': 'POSITIVE', 'score': 0.9998776316642761}]]
# pred=pipe(s)

# #np.argmax(pipe(s), axis=1).flatten()

In [57]:
# pred[0][1]

In [56]:
# !pip freeze > requirements.txt

In [55]:
# model=BertForSequenceClassification.from_pretrained('results/model/')
# tokenizer1 = BertTokenizer.from_pretrained("/content/results/tokenizer", local_files_only=True)

In [34]:
# from transformers import TextClassificationPipeline
# pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
# pred=pipe(s)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [35]:
pred

[[{'label': 'LABEL_0', 'score': 0.02043476514518261},
  {'label': 'LABEL_1', 'score': 0.16177953779697418},
  {'label': 'LABEL_2', 'score': 0.032369472086429596},
  {'label': 'LABEL_3', 'score': 0.00257749087177217},
  {'label': 'LABEL_4', 'score': 0.7828387022018433}]]

In [42]:
!zip -r /content/drive/MyDrive/result.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/tokenizer/ (stored 0%)
  adding: content/results/tokenizer/vocab.txt (deflated 53%)
  adding: content/results/tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/results/tokenizer/tokenizer_config.json (deflated 75%)
  adding: content/results/model/ (stored 0%)
  adding: content/results/model/config.json (deflated 53%)
  adding: content/results/model/model.safetensors (deflated 7%)


In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [53]:
# a=[
#     [
#         {
#             "label": "LABEL_0",
#             "score": 0.02043476141989231
#         },
#         {
#             "label": "LABEL_1",
#             "score": 0.16177958250045776
#         },
#         {
#             "label": "LABEL_2",
#             "score": 0.032369427382946014
#         },
#         {
#             "label": "LABEL_3",
#             "score": 0.0025774911046028137
#         },
#         {
#             "label": "LABEL_4",
#             "score": 0.782838761806488
#         }
#     ]
# ]

In [51]:
#a[0]

In [52]:
#max(a[0], key=lambda x:x['score'])["label"]